# Build a Sudoku Solver using just an array of sudoku

## Import all the necessary libraries

In [1]:
import string
import numpy as np
import random

## Define a Sudoku Class

In [ ]:
import numpy as np

class sudoku():
    def __init__(self, sudoku_array) -> None:
        self.n = len(sudoku_array)
        self.sqrt_n = int(np.sqrt(self.n))
        self.sudoku_original = sudoku_array
        self.sudoku_array = np.copy(sudoku_array)
        self.all_allowed_values = np.arange(1, self.n+1, 1)

        self.subgrids_array = self.get_subgrids()
        self.validate_n()
        # Get occupied values in each row and column
        self.rows_occupied = {i: [v for v in self.sudoku_array[i, :] if v > 0] for i in range(self.n)}
        self.cols_occupied = {i: [v for v in self.sudoku_array[:, i] if v > 0] for i in range(self.n)}
        self.rows_available = {i: [v for v in self.all_allowed_values if v not in self.rows_occupied[i]] for i in range(self.n)}
        self.cols_available = {i: [v for v in self.all_allowed_values if v not in self.cols_occupied[i]] for i in range(self.n)}
        self.available_cells = list(zip(*np.where(self.sudoku_array == 0)))

    def validate_n(self):
        if self.sqrt_n ** 2 != self.n:
            raise ValueError(f"Grid size {self.n} is not a perfect square.")
        if len(self.sudoku_array[:, 0]) != self.n:
            raise ValueError(f"Grid size {self.n} is not a perfect square.")
        for subgrid_num in range(len(self.subgrids_array)):
            subgrid_values = [v for v in self.subgrids_array[subgrid_num].flatten() if v > 0]  # ignore zeros
            if len(set(subgrid_values)) != len(subgrid_values):
                raise ValueError(f"Subgrid {subgrid_num} contains duplicate values.")

    def update_availability(self, x, y):
        self.subgrids_array = self.get_subgrids()
        # update only the affected row/col
        self.rows_occupied[x] = [v for v in self.sudoku_array[x, :] if v > 0]
        self.cols_occupied[y] = [v for v in self.sudoku_array[:, y] if v > 0]
        self.rows_available[x] = [v for v in self.all_allowed_values if v not in self.rows_occupied[x]]
        self.cols_available[y] = [v for v in self.all_allowed_values if v not in self.cols_occupied[y]]
        self.available_cells = list(zip(*np.where(self.sudoku_array == 0)))

    def insert_value(self, row, col, value, verbose):
        self.sudoku_array[row][col] = value
        self.update_availability(row, col)
        if verbose:
            print(f"Inserted {value} in row = {row}, column = {col}")

    # Function to cut the grid into nxn subgrids
    def get_subgrids(self):
        subgrids = []
        for row in range(0, self.n, self.sqrt_n):
            for col in range(0, self.n, self.sqrt_n):
                subgrid = self.sudoku_array[row:row+self.sqrt_n, col:col+self.sqrt_n]
                subgrids.append(subgrid)
        return subgrids

    def identify_subgrid(self, row, col):
        subgrid_row = row // self.sqrt_n
        subgrid_col = col // self.sqrt_n
        subgrid_number = subgrid_row * self.sqrt_n + subgrid_col
        return subgrid_number

    def subgrid_features(self, row, col):
        subgrid_num = self.identify_subgrid(row, col)
        occupied_integers = [i for i in self.subgrids_array[subgrid_num].flatten() if i > 0]
        available_integers = [i for i in self.all_allowed_values if i not in occupied_integers]
        return occupied_integers, available_integers

    def solve_cell_wise(self, row, col, verbose):
        cell_row_available = self.rows_available[row]
        cell_col_available = self.cols_available[col]
        cell_subgrid_available = self.subgrid_features(row, col)[1]
        possible_values = list(set(cell_row_available).intersection(cell_col_available).intersection(cell_subgrid_available))
        if len(possible_values) == 1:
            self.insert_value(row, col, possible_values[0], verbose)
            return True
        return False

    def subgrid_wise(self, grid_iter, verbose):
        row_idx = grid_iter // self.sqrt_n
        col_idx = grid_iter % self.sqrt_n
        list_rows = np.arange(row_idx*self.sqrt_n, (row_idx+1)*self.sqrt_n)
        list_columns = np.arange(col_idx*self.sqrt_n, (col_idx+1)*self.sqrt_n)
        list_values_to_be_filled_in = [i for i in self.all_allowed_values if i not in self.subgrids_array[grid_iter].flatten()]

        progress = False
        for num in list_values_to_be_filled_in:
            list_available_moves = []
            for i in list_rows:
                if num in self.rows_available[i]:
                    for j in list_columns:
                        if num in self.cols_available[j]:
                            if self.sudoku_array[i][j] == 0:
                                list_available_moves.append([i, j])
            if len(list_available_moves) == 1:
                i, j = list_available_moves[0]
                self.insert_value(i, j, num, verbose)
                progress = True
        return progress

    def print_sudoku(self):
        for i in range(self.n):
            if i % self.sqrt_n == 0 and i != 0:
                print("-" * (self.sqrt_n * 8))
            for j in range(self.n):
                if j % self.sqrt_n == 0 and j != 0:
                    print(" | ", end="")
                if self.sudoku_array[i][j] == 0:
                    cell_value = "."
                else:
                    cell_value = self.sudoku_array[i][j]
                if j == self.n-1:
                    print(cell_value)
                else:
                    print(f"{cell_value} ", end="")

    def solve_sudoku(self, verbose = False):
        progress = True
        while progress:
            progress = False
            for row, col in list(self.available_cells):  # copy to avoid mutation issues
                if self.solve_cell_wise(row, col, verbose):
                    progress = True
            for grid_iter in range(len(self.subgrids_array)):
                if self.subgrid_wise(grid_iter, verbose):
                    progress = True

        if len(self.available_cells) > 0:
            print("Couldn't fully solve the Sudoku using human methods, trying brute force")
            print("Current Sudoku state is: ")
            self.print_sudoku()
            self.backtrack_solve(verbose)
            if len(self.available_cells) > 0:
                print("Couldn't fully solve the Sudoku!")
            print("Sudoku solved with backtracking!")
        else:
            print("Sudoku solved!")


    def backtrack_solve(self, verbose):
        if len(self.available_cells) == 0:
            return True  # all cells filled

        # Pick the first empty cell
        row, col = self.available_cells[0]

        # Try all possible values for that cell
        _, possible_values = self.subgrid_features(row, col)
        for value in possible_values:
            # check row/col availability
            if value in self.rows_available[row] and value in self.cols_available[col]:
                # place value
                self.insert_value(row, col, value, verbose)
                if self.backtrack_solve(verbose):  # recursive call
                    return True
                # undo placement (backtrack)
                self.sudoku_array[row][col] = 0
                self.update_availability(row, col)

        return False  # no valid value found, backtrack

## Model configuration: Give a sudoku grid as an input

### Unique Solution

In [67]:
sudoku_grid = np.array([
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9],
])

In [68]:
n_sudoku = sudoku(sudoku_grid)

In [69]:
n_sudoku.print_sudoku()

5 3 .  | . 7 .  | . . .
6 . .  | 1 9 5  | . . .
. 9 8  | . . .  | . 6 .
------------------------
8 . .  | . 6 .  | . . 3
4 . .  | 8 . 3  | . . 1
7 . .  | . 2 .  | . . 6
------------------------
. 6 .  | . . .  | 2 8 .
. . .  | 4 1 9  | . . 5
. . .  | . 8 .  | . 7 9


In [70]:
n_sudoku.solve_sudoku(verbose=False)

Sudoku solved!


In [71]:
n_sudoku.print_sudoku()

5 3 4  | 6 7 8  | 9 1 2
6 7 2  | 1 9 5  | 3 4 8
1 9 8  | 3 4 2  | 5 6 7
------------------------
8 5 9  | 7 6 1  | 4 2 3
4 2 6  | 8 5 3  | 7 9 1
7 1 3  | 9 2 4  | 8 5 6
------------------------
9 6 1  | 5 3 7  | 2 8 4
2 8 7  | 4 1 9  | 6 3 5
3 4 5  | 2 8 6  | 1 7 9


### No unique solution

In [62]:
sudoku_grid = np.array([
    [5, 7, 0, 6, 9, 0, 0, 0, 0],
    [0, 8, 0, 2, 3, 1, 4, 7, 5],
    [4, 3, 0, 0, 0, 0, 0, 0, 0],
    [9, 0, 6, 1, 4, 0, 7, 0, 3],
    [0, 0, 0, 0, 8, 0, 0, 0, 0],
    [8, 0, 3, 0, 5, 9, 2, 0, 6],
    [0, 0, 0, 0, 0, 0, 0, 4, 7],
    [3, 9, 5, 4, 1, 7, 0, 6, 0],
    [0, 0, 0, 0, 2, 8, 0, 9, 1]
])

In [63]:
n_sudoku = sudoku(sudoku_grid)

In [64]:
n_sudoku.print_sudoku()

5 7 .  | 6 9 .  | . . .
. 8 .  | 2 3 1  | 4 7 5
4 3 .  | . . .  | . . .
------------------------
9 . 6  | 1 4 .  | 7 . 3
. . .  | . 8 .  | . . .
8 . 3  | . 5 9  | 2 . 6
------------------------
. . .  | . . .  | . 4 7
3 9 5  | 4 1 7  | . 6 .
. . .  | . 2 8  | . 9 1


In [65]:

n_sudoku.solve_sudoku(verbose = False)

Couldn't fully solve the Sudoku using human methods, trying brute force
Current Sudoku state is: 
5 7 2  | 6 9 4  | 1 3 8
6 8 9  | 2 3 1  | 4 7 5
4 3 1  | 8 7 5  | 6 2 9
------------------------
9 5 6  | 1 4 2  | 7 8 3
. . 7  | 3 8 6  | 9 5 4
8 4 3  | 7 5 9  | 2 1 6
------------------------
. . 8  | 9 6 3  | 5 4 7
3 9 5  | 4 1 7  | 8 6 2
7 6 4  | 5 2 8  | 3 9 1
Sudoku solved with backtracking!


In [66]:
n_sudoku.print_sudoku()

5 7 2  | 6 9 4  | 1 3 8
6 8 9  | 2 3 1  | 4 7 5
4 3 1  | 8 7 5  | 6 2 9
------------------------
9 5 6  | 1 4 2  | 7 8 3
1 2 7  | 3 8 6  | 9 5 4
8 4 3  | 7 5 9  | 2 1 6
------------------------
2 1 8  | 9 6 3  | 5 4 7
3 9 5  | 4 1 7  | 8 6 2
7 6 4  | 5 2 8  | 3 9 1
